
### Asserts

Asserts can be globally disabled with an interpreter settings
They shouldn't be used for security checks etc.

---

### Lists

Changes look better in commits with lists in separate lines.
Also put comma behind last item to make it easier to add new items.


This is better

In [ ]:
names = [
    'bob',
    'john',
    'robert',
]

Than this

In [ ]:
names = ['bob','john','robert']

And this

In [ ]:
names = [
    'bob',
    'john',
    'robert'
]

--- 
### Context managers

Context managers are classes with `__enter__` and `__exit__`.

In [27]:

class ContextManager:
    
    def __enter__(self):
        print('enter')
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('exit')
     

In [28]:
with ContextManager():
    print('hello')

enter
hello
exit


In [29]:
with ContextManager():
    raise ValueError

enter
exit


ValueError: 

--- 

### Underscores

### `_var` - Single leading underscore  
Private variable. This is just a convention (in PEP8)  
Python will not import import modules with leading underscores automatically.
Unless defined in `__all__`

### `var_` - Single trailing underscore  
If is variable name is already taken, use this.
`set_`,`class_`,`dict_` etc.
This is just a convention.

### `__var` - Double leading underscore  
Used for name mangling.  
Interpreter changes this name to something else, to prevent collision when extended.
 

In [26]:
class TestA:
    def __init__(self):
        self.a = 1
        self._a = 1
        self.__a = 1
        
    def do_something(self):
        print(self.a)
        print(self._a)
        print(self.__a)
        
tsta = TestA()
print(dir(tsta))
tsta.do_something()

# print(tsta.a)
# print(tsta._a)
# print(tsta.__a)
# This explodes -> __a is not here

class TestB(TestA):
    def __init__(self):
        
        super().__init__()
        
        self.a = 3
        self._a = 3
        self.__a = 3

tstb = TestB()
print(dir(tstb))
tstb.do_something()


['_TestA__a', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_a', 'a', 'do_something']
1
1
1
['_TestA__a', '_TestB__a', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_a', 'a', 'do_something']
3
3
1



### `__var__` - Double leading and training underscores  

### `_` - Single underscore  